<a href="https://colab.research.google.com/github/mangalm96/explainablenlp-info259/blob/main/interpret_shap_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import json
import pandas as pd

In [38]:
def generate_shap_df(filepath):
  with open(filepath) as f:
    lines = f.readlines()
  shap_df  = pd.DataFrame()
  for line in lines[:1]:
    dictionary = dict()
    if line != '':
      line_data = json.loads(line)
      # print(line_data.keys())
      print(line_data["tokens"])
      line_data_df = pd.DataFrame.from_dict(line_data["tokens"])
      # print(line_data_df)
      if len(shap_df) == 0:
        shap_df = line_data_df
      else:
        shap_df.append(line_data_df)
    # shap_df = shap_df.sort_values("token")
  return shap_df

In [39]:
CI_ESRX_target = "/content/gdrive/MyDrive/NLP Stance Detection/xai_atanasova/WTWT/CI_ESRX_target/model_output_shap"
shap_CI_ESRX = generate_shap_df(CI_ESRX_target)

[{'token': ['<s>'], '0': -0.01468949019908905, '1': -0.03274604305624962, '2': -0.013661552220582962, '3': 0.019281791523098946}, {'token': ['<unk>'], '0': -0.06967433542013168, '1': -0.02248510718345642, '2': 0.023149508982896805, '3': 0.019069842994213104}, {'token': ['and'], '0': -0.01549686398357153, '1': -0.10564021021127701, '2': 0.37225571274757385, '3': -0.0678384080529213}, {'token': ['<unk>'], '0': -0.03128916770219803, '1': -0.20271623134613037, '2': 0.1874406784772873, '3': -0.05329563096165657}, {'token': ['set'], '0': 0.01151291560381651, '1': -0.18747073411941528, '2': 0.13602198660373688, '3': -0.055543262511491776}, {'token': ['to'], '0': -0.13459984958171844, '1': -0.39619895815849304, '2': 0.2721729576587677, '3': -0.11129413545131683}, {'token': ['merge'], '0': -0.30935004353523254, '1': -0.11708498001098633, '2': 0.13065868616104126, '3': -0.10616012662649155}, {'token': ['.'], '0': -0.019403640180826187, '1': -0.04806576669216156, '2': 0.23945504426956177, '3': -0

In [ ]:
[{'token': ['<s>'], '0': -0.003145422786474228, '1': 0.015037721022963524, '2': 0.0011098348768427968, '3': 0.012733332812786102},
 {'token': ['<unk>'], '0': -0.05680133029818535, '1': -0.04227682203054428, '2': -0.02990284375846386, '3': 0.061003729701042175},
 {'token': ['profits'], '0': 0.08510257303714752, '1': -0.13474957644939423, '2': 0.08724816143512726, '3': -0.08916759490966797},
 {'token': ['jump'], '0': 0.20763686299324036, '1': -0.12225738167762756, '2': 0.11071815341711044, '3': -0.015009257011115551},
 {'token': ['amid'], '0': 0.13723354041576385, '1': -0.4388749599456787, '2': -0.27289265394210815, '3': 0.22624839842319489},
 {'token': ['<unk>'], '0': -0.030833758413791656, '1': -0.004953343886882067, '2': -0.08957904577255249, '3': 0.12224700301885605}, {'token': ['about'], '0': 0.08810034394264221, '1': -0.12134896963834763, '2': -0.025922946631908417, '3': 0.16283255815505981}, {'token': ['Express'], '0': 0.06495410948991776, '1': 0.09127680212259293, '2': -0.09509134292602539, '3': 0.030929889529943466}, {'token': ['<unk>'], '0': 0.12362872809171677, '1': -0.19919805228710175, '2': 0.03876568377017975, '3': 0.02693309634923935}, {'token': ['deal'], '0': 0.189987450838089, '1': -0.8205814957618713, '2': 0.10457111150026321, '3': 0.18607182800769806}, {'token': ['by'], '0': -0.011395675130188465, '1': -0.019954878836870193, '2': 0.06972026079893112, '3': 0.06547949463129044}, {'token': ['<unk>'], '0': 0.007005555555224419, '1': -0.22459106147289276, '2': -0.06533335894346237, '3': 0.01890786923468113}, {'token': ['</s>'], '0': 0.025558093562722206, '1': -0.061034511774778366, '2': -0.020407194271683693, '3': 0.04838964343070984}, {'token': ['</s>'], '0': 0.0020118379034101963, '1': 0.08336425572633743, '2': 0.07613315433263779, '3': 0.0006143403006717563}, {'token': ['Cig@@'], '0': -0.010051733814179897, '1': 0.02282833494246006, '2': 0.02313261665403843, '3': -0.021204279735684395}, {'token': ['na'], '0': -0.05272137001156807, '1': 0.05922555923461914, '2': -0.016753453761339188, '3': 0.012517912313342094}, {'token': ['Ex@@'], '0': 0.03229968622326851, '1': -0.15136295557022095, '2': 0.10724681615829468, '3': 0.02799166366457939}, {'token': ['pres@@'], '0': 0.024911150336265564, '1': -0.020287007093429565, '2': 0.051607709378004074, '3': 0.011440581642091274}, {'token': ['s_@@'], '0': 0.03538767620921135, '1': -0.04082281515002251, '2': 0.11994462460279465, '3': 0.04171360284090042}, {'token': ['Scri@@'], '0': -0.0007254814845509827, '1': -0.012289213947951794, '2': -0.04084115847945213, '3': -0.0733000785112381}, {'token': ['pts'], '0': -0.002446749247610569, '1': 0.006674897391349077, '2': 0.0985182374715805, '3': -0.01890104077756405}, {'token': ['</s>'], '0': -0.02156221494078636, '1': 0.07741747051477432, '2': 0.02185642160475254, '3': -0.013030006550252438}]


In [ ]:
shap_CI_ESRX.head(30)

,token,0,1,2,3
0,[<s>],-0.014689,-0.032746,-0.013662,0.019282
1,[<unk>],-0.069674,-0.022485,0.023150,0.019070
2,[and],-0.015497,-0.105640,0.372256,-0.067838
3,[<unk>],-0.031289,-0.202716,0.187441,-0.053296
4,[set],0.011513,-0.187471,0.136022,-0.055543
5,[to],-0.134600,-0.396199,0.272173,-0.111294
6,[merge],-0.309350,-0.117085,0.130659,-0.106160
7,[.],-0.019404,-0.048066,0.239455,-0.043079
8,[Here],-0.065872,-0.146163,0.165911,-0.130518
9,[we],-0.088377,-0.107532,0.007042,-0.068374


In [ ]:
CI_ESRX_notarget = "/content/gdrive/MyDrive/NLP Stance Detection/xai_atanasova/WTWT/CI_ESRX_notarget/model_output_shap"
shap_CI_ESRX_notarget = generate_shap_df(CI_ESRX_notarget)

In [ ]:
bshap_CI_ESRX_notarget.head(30)

,token,0,1,2,3
11,[.],-0.060951,0.058714,0.192585,-0.020560
7,[.],-0.019404,-0.048066,0.239455,-0.043079
21,[</s>],-0.049227,0.084209,0.183739,-0.013249
13,[</s>],-0.068591,-0.090330,0.166975,-0.046177
12,[</s>],-0.075020,-0.079719,0.015410,-0.046493
0,[<s>],-0.014689,-0.032746,-0.013662,0.019282
3,[<unk>],-0.031289,-0.202716,0.187441,-0.053296
1,[<unk>],-0.069674,-0.022485,0.023150,0.019070
14,[Cig@@],-0.014407,-0.030730,0.310898,-0.039025
16,[Ex@@],-0.069368,-0.068314,0.088047,-0.011514


## on a sentence

In [44]:
def generate_shap_df(filepath):
  with open(filepath) as f:
    lines = f.readlines()
  shap_df  = pd.DataFrame()
  for line in lines[3:4]:
    dictionary = dict()
    if line != '':
      line_data = json.loads(line)
      # print(line_data.keys())
      # print(line_data["tokens"])
      line_data_df = pd.DataFrame.from_dict(line_data["tokens"])
      # print(line_data_df)
      if len(shap_df) == 0:
        shap_df = line_data_df
      else:
        shap_df.append(line_data_df)
    # shap_df = shap_df.sort_values("token")
  return shap_df

In [44]:
# {'comment': 0, 'unrelated': 1, 'support': 2, 'refute': 3}

In [45]:
CI_ESRX_notarget = "/content/gdrive/MyDrive/NLP Stance Detection/xai_atanasova/WTWT/CI_ESRX_notarget/model_output_shap"

shap_CI_ESRX_notarget = generate_shap_df(CI_ESRX_notarget)

         token         0         1         2         3
0        [<s>] -0.007589  0.017476  0.017817 -0.006708
1       [Here]  0.103325 -0.172322 -0.038915  0.004067
2          [']  0.010507 -0.081621  0.021931 -0.027868
3          [s] -0.055271  0.172182 -0.079059 -0.081908
4        [the]  0.175509 -0.247975 -0.062219  0.021631
5       [just]  0.086136  0.133806 -0.025826 -0.056298
6          [-]  0.019058 -0.040276 -0.055737 -0.016335
7   [released] -0.038534  0.148815 -0.033595 -0.046920
8   [<number>] -0.225215  0.205150 -0.103850 -0.023879
9          [+]  0.040828  0.014214 -0.088289 -0.029222
10      [page] -0.251387  0.701718 -0.201419 -0.055718
11    [merger]  0.440218 -0.530476  0.314148 -0.137285
12     [proxy] -0.288285  0.714136 -0.212534 -0.071578
13         [S] -0.091201  0.247573 -0.095807 -0.035357
14         [-]  0.222338 -0.117032 -0.019024  0.009538
15  [<number>] -0.072895  0.021319 -0.054954 -0.037351
16    [filing]  0.218220 -0.342469  0.040240  0.019196
17      [f

In [59]:
shap_CI_ESRX_notarget

,token,0,1,2,3
0,[<s>],-0.007589,0.017476,0.017817,-0.006708
1,[Here],0.103325,-0.172322,-0.038915,0.004067
2,['],0.010507,-0.081621,0.021931,-0.027868
3,[s],-0.055271,0.172182,-0.079059,-0.081908
4,[the],0.175509,-0.247975,-0.062219,0.021631
5,[just],0.086136,0.133806,-0.025826,-0.056298
6,[-],0.019058,-0.040276,-0.055737,-0.016335
7,[released],-0.038534,0.148815,-0.033595,-0.046920
8,[<number>],-0.225215,0.205150,-0.103850,-0.023879
9,[+],0.040828,0.014214,-0.088289,-0.029222


In [70]:
# rename cols
shap_CI_ESRX_notarget = shap_CI_ESRX_notarget.rename(columns = {"0":"comment","1":"unrelated","2":"support","3":"refute"})

In [90]:
shap_CI_ESRX_notarget['token'] = shap_CI_ESRX_notarget['token'].astype(str)

In [79]:
tt =  shap_CI_ESRX_notarget[['token','comment']]

In [94]:
# group by token and sum for each column
shap_CI_ESRX_notarget.groupby('token').agg('mean')


,comment,unrelated,support,refute
token,,,,
"[""'""]",0.010507,-0.081621,0.021931,-0.027868
['!'],-0.058961,0.064751,-0.035932,-0.026096
['$'],0.229970,-0.195008,0.099741,-0.077173
['+'],0.040828,0.014214,-0.088289,-0.029222
['-'],0.120698,-0.078654,-0.037381,-0.003399
['.'],0.032682,-0.040758,-0.002568,-0.029764
['</s>'],0.031306,0.025904,-0.054313,-0.059042
['<number>'],-0.149055,0.113234,-0.079402,-0.030615
['<s>'],-0.007589,0.017476,0.017817,-0.006708


## import predictions

In [54]:
predictions_path = "/content/gdrive/MyDrive/NLP Stance Detection/Code_Kaushal/saves/wtwt_CI_ESRX_target/model.predictions"

with open(predictions_path, 'r') as f:
  data = json.load(f)


In [58]:
data['class'][:4]

[2, 2, 0, 0]

In [53]:
# import predictions
predictions_path = "/content/gdrive/MyDrive/NLP Stance Detection/Code_Kaushal/saves/wtwt_CI_ESRX_target/model.predictions"

def generate_shap_df(filepath):
  line_data = json.loads(line)
  with open(filepath) as f:
    data = json.load(f)
  print(data)

  # shap_df  = pd.DataFrame()
  # for line in lines:
  #   print(line)
  #   # dictionary = dict()
  #   if line != '':
  #     line_data = json.loads(line)
  #     # print(line_data.keys())
  #     print(line_data)
  #     line_data_df = pd.DataFrame.from_dict(line_data["tokens"])
  #     # print(line_data_df)
  #     if len(shap_df) == 0:
  #       shap_df = line_data_df
  #     else:
  #       shap_df.append(line_data_df)
  #   # shap_df = shap_df.sort_values("token")
  # return shap_df